## Import Libraries

In [1]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image

In [2]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

In [124]:
fileList = glob.glob("./Images/*.*")

In [ ]:
directory = "Male"
    
if not os.path.exists(directory):
    os.makedirs(directory)

directory1 = "Female"

if not os.path.exists(directory1):
    os.makedirs(directory1)    

    
for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            new_file_extension = prof['Gender']
            new_file_extension = new_file_extension.title()
            #os.rename(fp, filename+"."+new_file_extension)
            copyfile(filename+".jpg", './Lable_Images1/'+ uid + '.' + str(id) + "."+new_file_extension+'.jpg')
            #copyfile(filename+".jpg", new_file_extension +'/'+uid+"."+new_file_extension)

In [ ]:
import os
for file in os.listdir("Lable_Images1"):
    file_path = os.path.join("Lable_Images1", file)
    try:
        if not file.endswith('.jpg'):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [3]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [13]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    gender =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        gender_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                images.append(resized)
                labels.append(nbr)

                if gender_current == 'Male':
                    gender.append(1)
                else:
                    gender.append(0)

                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(50)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print gender
    
    return images, labels, gender

In [78]:
images, labels, gender = get_images_and_labels('Lable_Images1')

cv2.destroyAllWindows()

701
1019
730
587
390
588
900
1018
396
887
1173
430
1076
49
780
961
571
496
965
647
77
378
210
983
132
397
81
363
1033
193
445
687
417
1080
1079
93
676
466
739
487
416
987
434
313
1117
114
164
778
471
959
699
41
284
1090
273
419
538
1166
1108
750
744
1066
1036
1035
525
1109
796
1068
451
152
964
161
460
1037
1120
826
1100
654
399
189
446
1046
404
306
209
945
83
767
1136
664
988
650
625
928
989
109
108
723
657
213
264
427
301
1089
612
1016
1121
985
62
178
721
764
876
320
846
194
295
726
23
212
151
394
156
249
1015
492
228
711
97
241
1130
325
470
968
1010
115
670
265
560
848
909
237
1103
339
300
290
656
986
1145
1027
107
355
698
745
729
1097
578
106
845
595
103
1058
854
828
883
134
1161
457
692
757
188
1072
142
1168
409
1044
856
901
1083
747
160
80
785
346
552
537
424
892
304
260
225
759
517
1078
519
580
177
1158
458
24
495
641
820
414
1086
627
877
253
1167
1087
749
691
873
954
11
798
170
797
319
779
251
1155
535
748
342
534
1062
616
832
379
1092
546
205
831
46
863
722
896
1149
570
867
626

In [80]:
images[0].shape

(100, 100)

## Making the Dataset 

In [119]:
res_images = []
res_gender = []

for img in images:
    res_images.append(img.reshape(1,10000))    # (100*100 image size)

for gen in gender:
    res_gender.append(np.array(gen).reshape(1,))
    
res_images = np.concatenate(res_images, axis=0)
res_gender = np.concatenate(res_gender,axis=0)

In [122]:
res_images.shape

(1032, 10000)

In [123]:
res_gender.shape

(1032,)

In [113]:
X,y = res_images,res_gender

In [117]:
X_Train, X_Test = X[:1000], X[1000:]
y_Train, y_Test = y[:1000], y[1000:]


print y_Train.shape,y_Test.shape

(1000,) (32,)


In [118]:
# Logistic Regression
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# load the iris datasets
#dataset = datasets.load_iris()
# fit a logistic regression model to the data
model = LogisticRegression(C=1e5)
model.fit(X_Train,y_Train)
print(model)
# make predictions
expected = y_Test
predicted = model.predict(X_Test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.64      0.75      0.69        12
          1       0.83      0.75      0.79        20

avg / total       0.76      0.75      0.75        32

[[ 9  3]
 [ 5 15]]
